## Twitter Data
#### *Skip until load pickle for retrieved data.

In [ ]:
# import all neccessary libraries
from datetime import datetime
import json
import geocoder as geo
import glob
import tweepy as tw
import pandas as pd
import pickle

pd.set_option('max_colwidth',150)

In [ ]:
# Twitter API keys
consumer_key = ''
consumer_secret = ''
access_key = ''
access_secret = ''

# get authentication
auth = tw.OAuth1UserHandler(consumer_key, consumer_secret, access_key, access_secret)
api = tw.API(auth)

In [ ]:
# get location
location = 'Malaysia'
geo_osm = geo.osm(location)

# get trends
closest_loc = api.closest_trends(geo_osm.lat, geo_osm.lng)
trends = api.get_place_trends(closest_loc[0]['woeid'])[0]['trends']

# get trend hashtags
hashtags = []
[hashtags.append(trend['name']) for trend in trends]
    
    
print(hashtags)

In [ ]:
# get Malaysia place id for Twitter
places = api.search_geo(query='Malaysia', granularity='country')
place_id = places[0].id

# retrieve tweet data by respective hashtag
tweet_data = []
try:
    for hashtag in hashtags:
        tweets = tw.Cursor(api.search_tweets, q='%s place:%s' % (hashtag, place_id)).items(100)
    
        for tweet in tweets:
            tweet_data.append({
                'date': tweet.created_at,
                'id': tweet.id,
                'hashtag': hashtag,
                'text': tweet.text,
            })
finally:
    # store data in form of pickle for later use
    !mkdir data

    time_now = datetime.now().strftime("%d-%m-%Y %H-%M-%S")

    with open('data-malaysian/twitter/twitter %s.pickle' % time_now, 'wb') as file:
        pickle.dump(tweet_data, file)

In [ ]:
# load Twitter pickle file
combined_tweets = []

# get all tweets
tweets_directories = glob.glob('data-malaysian/twitter/twitter*')
for directory in tweets_directories:
    with open(directory, 'rb') as file:
        combined_tweets += pickle.load(file)

# shows data in Dataframe form
tweets_df = pd.DataFrame.from_dict(combined_tweets)
tweets_df